In [186]:
from t2_flujo import *
import folium
import matplotlib.pyplot as plt
import funciones
import parametros

In [187]:
N = []
for a in range(len(nodos_totales)):
    lista = nodos_totales.loc[a].to_list()
    nodo = funciones.Node(lista[0], lista[1], lista[2], lista[3], lista[4])
    N.append(nodo)

# DICCIONARIO DE ID EN EL MAPA Y EL ID QUE LE PUSE A LOS NODOS

ids = nodos_totales['id_nodo'].to_list()
nombres = nodos_totales['id'].to_list()

dict_ids_nombres = {}

for a in range(len(ids)):
    dict_ids_nombres[ids[a]] = nombres[a]

for a in range(1,parametros.P+1):
    nombres.append(f'PB{a}')

In [188]:
# Como se especifica que el transporte es desde los nodos de produccion a bodegas y demanda, y de los nodos
# de bodega a los nodos de demanda, elimino los arcos que no sirven para del grafo de todos los nodos conectados entre si.

# list(grafo_vial.edges(data=True))[par][origen/destino]
grafo_vial_nuevo = grafo_vial.copy()
lista_pares_nodos = list(grafo_vial_nuevo.edges())

for b in range(len(lista_pares_nodos)):
    a = lista_pares_nodos[b]
    
    O = nodos_totales[nodos_totales['id_nodo'] == a[0]]
    O_pos = O.index[0]

    D = nodos_totales[nodos_totales['id_nodo'] == a[1]]
    D_pos = D.index[0]

    if O['tipo'][O_pos] == D['tipo'][D_pos]:
        grafo_vial_nuevo.remove_edge(O['id_nodo'][O_pos],D['id_nodo'][D_pos])
    
    elif O['tipo'][O_pos] == 'demanda':
        grafo_vial_nuevo.remove_edge(O['id_nodo'][O_pos], D['id_nodo'][D_pos])

info_tipo_nodo = list(dict(grafo_vial_nuevo.nodes(data=True)).values())

In [221]:
# Las rutas minimas entre los nodos (todo el grafo se demora 1 min aprox)
r_min = funciones.rutas_min(grafo_vial_nuevo, parametros.G, len(list(grafo_vial_nuevo.edges())))
#graficar_rutas_min_n(r_min[0], G, 100)

In [222]:
# matriz de costos con las distancias entre los nodos en el mapa
matriz_costos = []
lista_arcos = list(grafo_vial_nuevo.edges(data=True))

for a in range(len(r_min[0])):
    matriz_costos.append([dict_ids_nombres[lista_arcos[a][0]],
                         dict_ids_nombres[lista_arcos[a][1]],
                         r_min[1][a],
                         r_min[0][a]])

In [225]:
matriz_costos_prod_dem = pd.DataFrame(columns=['origen', 'destino', 'distancia', 'ruta'])
matriz_costos_prod_bod = pd.DataFrame(columns=['origen', 'destino', 'distancia', 'ruta'])
matriz_costos_bod_dem = pd.DataFrame(columns=['origen', 'destino', 'distancia', 'ruta'])

for a in matriz_costos:
    if 'P' in a[0]:
        if 'D' in a[1]:
            matriz_costos_prod_dem = matriz_costos_prod_dem._append({'origen':a[0], 'destino':a[1], 'distancia':a[2], 'ruta':a[3]}, ignore_index=True)
        elif 'B' in a[1]:
            matriz_costos_prod_bod = matriz_costos_prod_bod._append({'origen':a[0], 'destino':a[1], 'distancia':a[2], 'ruta':a[3]}, ignore_index=True)
    if 'B' in a[0]:
        matriz_costos_bod_dem = matriz_costos_bod_dem._append({'origen':a[0], 'destino':a[1], 'distancia':a[2], 'ruta':a[3]}, ignore_index=True)

In [234]:
I = parametros.P
J = parametros.B + 1

# PARÁMETROS COMUNES ENTRE LA MATRIZ DE VIAJES Y COSTOS
columnas = list(nodos_bodegas.index)
filas = []
for a in range(1,I):
    columnas.append(a)
    filas.append(np.float64(0))
filas.append(np.float64(0))

dict_matriz = dict()
for b in range(0,J-1):
    dict_matriz[columnas[b]] = filas

prod_bodega = pd.DataFrame(data = dict_matriz)


prod_bodega['id'] = list(nodos_produccion.index)

prod_bodega = prod_bodega.set_index('id')

for i in range(len(matriz_costos_prod_bod)):
    fila = matriz_costos_prod_bod.loc[i]['origen']
    columna = matriz_costos_prod_bod.loc[i]['destino']
    guardar = matriz_costos_prod_bod.loc[i]['distancia']
    prod_bodega.loc[fila][columna] = guardar

In [239]:
I = parametros.P
J = parametros.D + 1

# PARÁMETROS COMUNES ENTRE LA MATRIZ DE VIAJES Y COSTOS
columnas = list(nodos_demanda.index)
filas = []
for a in range(1,I):
    columnas.append(a)
    filas.append(np.float64(0))
filas.append(np.float64(0))

dict_matriz = dict()
for b in range(0,J-1):
    dict_matriz[columnas[b]] = filas

prod_demanda = pd.DataFrame(data = dict_matriz)


prod_demanda['id'] = list(nodos_produccion.index)

prod_demanda = prod_demanda.set_index('id')

for i in range(len(matriz_costos_prod_dem)):
    fila = matriz_costos_prod_dem.loc[i]['origen']
    columna = matriz_costos_prod_dem.loc[i]['destino']
    guardar = matriz_costos_prod_dem.loc[i]['distancia']
    prod_demanda.loc[fila][columna] = guardar

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,D15,D16,D17,D18,D19,D20,D21,D22,D23,D24
id,,,,,,,,,,,,,,,,,,,,,
P1,33.0,47.0,85.0,60.0,17.0,81.0,32.0,68.0,26.0,73.0,...,69.0,42.0,60.0,21.0,75.0,77.0,14.0,27.0,83.0,33.0
P2,33.0,58.0,104.0,71.0,19.0,100.0,43.0,84.0,33.0,77.0,...,80.0,53.0,71.0,21.0,94.0,96.0,36.0,29.0,102.0,55.0
P3,60.0,41.0,48.0,25.0,64.0,38.0,52.0,30.0,62.0,53.0,...,32.0,39.0,23.0,64.0,26.0,40.0,37.0,63.0,40.0,44.0
P4,61.0,42.0,42.0,22.0,65.0,24.0,53.0,7.0,63.0,40.0,...,27.0,40.0,20.0,65.0,26.0,34.0,38.0,64.0,26.0,45.0
P5,46.0,27.0,44.0,20.0,51.0,40.0,38.0,27.0,46.0,39.0,...,29.0,21.0,20.0,51.0,34.0,36.0,24.0,50.0,42.0,31.0
P6,59.0,73.0,111.0,86.0,54.0,107.0,58.0,94.0,52.0,99.0,...,95.0,68.0,86.0,54.0,101.0,103.0,34.0,53.0,109.0,43.0
P7,34.0,48.0,92.0,61.0,14.0,88.0,33.0,74.0,27.0,74.0,...,70.0,43.0,61.0,12.0,82.0,84.0,23.0,27.0,90.0,42.0


In [242]:
I = parametros.B
J = parametros.D + 1

# PARÁMETROS COMUNES ENTRE LA MATRIZ DE VIAJES Y COSTOS
columnas = list(nodos_demanda.index)
filas = []
for a in range(1,I):
    columnas.append(a)
    filas.append(np.float64(0))
filas.append(np.float64(0))

dict_matriz = dict()
for b in range(0,J-1):
    dict_matriz[columnas[b]] = filas

bod_demanda = pd.DataFrame(data = dict_matriz)


bod_demanda['id'] = list(nodos_bodegas.index)

bod_demanda = bod_demanda.set_index('id')

for i in range(len(matriz_costos_bod_dem)):
    fila = matriz_costos_bod_dem.loc[i]['origen']
    columna = matriz_costos_bod_dem.loc[i]['destino']
    guardar = matriz_costos_bod_dem.loc[i]['distancia']
    bod_demanda.loc[fila][columna] = guardar

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,D15,D16,D17,D18,D19,D20,D21,D22,D23,D24
id,,,,,,,,,,,,,,,,,,,,,
B1,72.0,53.0,31.0,37.0,76.0,44.0,64.0,40.0,74.0,60.0,...,44.0,51.0,35.0,76.0,10.0,23.0,49.0,75.0,46.0,56.0
B2,36.0,27.0,68.0,29.0,61.0,46.0,28.0,37.0,45.0,23.0,...,24.0,25.0,25.0,57.0,58.0,60.0,45.0,49.0,48.0,52.0
B3,50.0,31.0,51.0,2.0,59.0,38.0,42.0,21.0,53.0,43.0,...,15.0,29.0,6.0,59.0,37.0,43.0,32.0,58.0,40.0,39.0
B4,32.0,41.0,72.0,48.0,28.0,68.0,31.0,55.0,25.0,62.0,...,57.0,32.0,48.0,27.0,62.0,64.0,21.0,26.0,70.0,40.0
B5,56.0,70.0,108.0,83.0,51.0,104.0,55.0,91.0,49.0,96.0,...,92.0,65.0,83.0,51.0,98.0,100.0,30.0,50.0,106.0,23.0
B6,39.0,57.0,96.0,70.0,16.0,92.0,42.0,79.0,36.0,83.0,...,79.0,52.0,70.0,20.0,86.0,88.0,25.0,34.0,94.0,44.0
B7,31.0,26.0,67.0,33.0,56.0,47.0,29.0,44.0,44.0,24.0,...,30.0,24.0,31.0,52.0,57.0,59.0,44.0,44.0,49.0,51.0
B8,24.0,28.0,78.0,44.0,42.0,64.0,4.0,56.0,21.0,41.0,...,42.0,29.0,43.0,38.0,68.0,70.0,39.0,31.0,66.0,58.0
B9,16.0,36.0,93.0,59.0,28.0,76.0,14.0,71.0,22.0,53.0,...,57.0,37.0,58.0,24.0,83.0,85.0,37.0,22.0,78.0,56.0


In [271]:
g = nx.DiGraph()

# NODOS
# 1. PLANTAS
#   (a) plantas - nodo auxiliar: recibe de las bodegas y de la producción,
#                                envía a nodo sumidero. 
#       P'i,t: i pertenece al conjunto nodos planta de producción,
#              t pertenece al conjunto de periodos de tiempo.

for t in range(1, parametros.T + 1):
    for p in range(1, parametros.P + 1):
        nodo = f'P auxiliar {p},{t}'
        g.add_node(nodo, demand = 0)

#   (b) plantas - producción: producen y "envían" a los nodos auxiliares de
#                             producción. 
#       P i,t: i pertenece al conjunto de nodos planta de producción,
#              t pertenece al conjunto de periodos de tiempo - {T}.
# NECESITAN ALGUN TIPO DE GENERACION
for t in range(1, parametros.T):
    for p in range(1, parametros.P + 1):
        nodo = f'P produccion {p},{t}'
        g.add_node(nodo, demand = 0)

#   (c) plantas - bodega: recibe la producción guardada de un periodo a otro,
#                         envia produccion a las bodegas auxiliares, demandas
#                         y nodo auxiliar de la planta (envía sobredemanda a
#                         nodo sumidero?)
#       PB i,t: i pertenece al conjunto de nodos planta de producción,
#               t pertenece al conjunto de periodo de timepo. 

for t in range(1, parametros.T + 1):
    for p in range(1, parametros.P + 1):
        nodo = f'P bodega {p},{t}'
        
        if t == 1:
            g.add_node(nodo, demand = inventario_inicial['1'].loc[f'P{p}'])
        else:
            g.add_node(nodo, demand = 0)

# 2. BODEGAS
#   (a) bodegas auxiliares: reciben inventario de las bodegas de las plantas
#                           de produccion, envian a los nodos de demanda
#                           (clientes).
#       B i,t: i pertence al conjunto de nodos de bodegas auxiliares, 
#              t pertenece al conjunto de periodos de tiempo.

for t in range(1, parametros.T + 1):
    for b in range(1, parametros.B + 1):
        nodo = f'B {b},{t}'

        if t == 1:
            g.add_node(nodo, demand = inventario_inicial_adicional['1'].loc[f'B{b}'])
        else:
            g.add_node(nodo, demand = 0)

# 3. CLIENTES
#   (a) clientes: reciben inventario desde las bodegas de las plantas de
#                 producción y bodegas adicionales.
#       D i,t: i pertenece al conjunto de nodos de clientes,
#              t pertenece al conjunto de periodos de tiempo.

for t in range(1, parametros.T + 1):
    for d in range(1, parametros.D + 1):
        nodo = f'D {d},{t}'
        g.add_node(nodo, demand = 0)

#   (b) clientes auxiliar: reciben desde el nodo de cliente, envía al nodos
#                          sumidero.
#       D' i,t: i pertenece al conjunto de nodos de clientes,
#               t pertenece al conjunto de periodos de tiempo.

for t in range(1, parametros.T + 1):
    for d in range(1, parametros.D + 1):
        nodo = f'D auxiliar {d},{t}'
        g.add_node(nodo, demand = 0)

# 4. SUMIDERO
#   (a) recibe las demandas y excesos de oferta, nodo atemporal.
#       S
g.add_node('s', demand = oferta_max_proveedores[['1','2','3','4','5','6','7']].sum().sum())

g.add_node('t', demand = -oferta_max_proveedores[['1','2','3','4','5','6','7']].sum().sum()
           - inventario_inicial_adicional['1'].sum()
           - inventario_inicial['1'].sum())


In [272]:
# ARCOS
# 1. Desde el nodo generador:
#    (s , (P i,t))

for t in range(1, parametros.T):
    for p in range(1, parametros.P + 1):
        origen = 's'
        destino = f'P produccion {p},{t}' 
        g.add_edge(origen,
                   destino,
                   cota_min = oferta_min_proveedores.loc[f'P{p}'][f'{t}'],
                   capacity = oferta_max_proveedores.loc[f'P{p}'][f'{t}'])

# 2. AL SUMIDERO:
#   (a) ((P i,t) , t):
for t in range(1, parametros.T):
    for p in range(1, parametros.P + 1):
        origen = f'P auxiliar {p},{t}' 
        destino = 't'
        g.add_edge(origen,
                   destino,
                   cota_min = 0)

#   (b) ((P' i,t) , t):
for t in range(1, parametros.T + 1):
    for p in range(1, parametros.P + 1):
        origen = 's'
        destino = f'P auxiliar {p},{t}' 
        g.add_edge(origen,
                   destino,
                   cota_min = 0)

#    (c) ((D' i,t) , t):
for t in range(1, parametros.T + 1):
    for d in range(1, parametros.D + 1):
        origen = 's'
        destino = f'D {d},{t}' 
        g.add_edge(origen,
                   destino,
                   cota_min = 0)
        
# 3. PRODUCTORES:
#   (a) ((P i,t) , (P' i,t))
for t in range(1, parametros.T):
    for p in range(1, parametros.P + 1):
        origen = f'P produccion {p},{t}'
        destino = f'P auxiliar {p},{t}'
        g.add_edge(origen,
                   destino,
                   #capacity = 0,
                   #cota_min = oferta_min_proveedores.loc[f'P{p}'][f'{t}'],
                   cota_min = 0,
                   weight = costo_produccion.loc[f'P{p}'][f'{t}'])
        
# 4. INVENTARIO:
#   (a) ((P bodega i,t) , (P' i,t))
for t in range(1, parametros.T + 1):
    for p in range(1, parametros.P + 1):
        origen = f'P bodega {p},{t}'
        destino = f'P auxiliar {p},{t}'
        g.add_edge(origen,
                   destino,
                   cota_min = 0,
                   weight = 0)

#   (b) ((P auxiliar i,t) , (P bodega i,t+1))
for t in range(1, parametros.T):
    for p in range(1, parametros.P + 1):
        origen = f'P auxiliar {p},{t}'
        destino = f'P bodega {p},{t+1}'
        g.add_edge(origen,
                   destino,
                   cota_min = 0,
                   capacity = capacidad_almacenaje.loc[f'P{p}'][f'{t}'],
                   weight = costo_almacenaje.loc[f'P{p}'][f'{t}'])

#   (c) ((B i,t) , (B i,t+1))
for t in range(1, parametros.T):
    for b in range(1, parametros.B + 1):
        origen = f'B {b},{t}'
        destino = f'B {b},{t+1}'
        g.add_edge(origen,
                   destino,
                   cota_min = 0,
                   capacity = capacidad_almacenaje_adicional.loc[f'B{b}'][f'{t}'],
                   weight = costo_almacenaje_adicional.loc[f'B{b}'][f'{t}'])

# DEMANDA:
#   (a) ((P bodega i,t) , (D i,t))
for t in range(1, parametros.T):
    for p in range(1, parametros.P + 1):
        for d in range(1, parametros.D + 1):
            origen = f'P bodega {p},{t}'
            destino = f'D {b},{t}'
            g.add_edge(origen,
                       destino,
                       cota_min = demanda_min.loc[f'D{b}'][f'{t}'],
                       capacity = demanda_max.loc[f'D{b}'][f'{t}'],
                       weight = prod_demanda.loc[f'P{p}'][f'D{d}'])

#   (b) ((B i,t) , (D i,t))
for t in range(1, parametros.T):
    for b in range(1, parametros.B + 1):
        for d in range(1, parametros.D + 1):
            origen = f'B {b},{t}'
            destino = f'D {b},{t}'
            g.add_edge(origen,
                       destino,
                       cota_min = demanda_min.loc[f'D{b}'][f'{t}'],
                       capacity = demanda_max.loc[f'D{b}'][f'{t}'],
                       weight = bod_demanda.loc[f'B{b}'][f'D{d}'])

In [273]:
for u,v,a in g.edges(data=True):
    if a["cota_min"] > 0:
        nx.set_node_attributes(g,{u:{'demand':g.nodes[u]['demand']+a['cota_min']}})
        nx.set_node_attributes(g,{v:{'demand':g.nodes[v]['demand']-a['cota_min']}})
        nx.set_edge_attributes(g,{(u,v):{'capacity':a['capacity']-a['cota_min']}})


In [274]:
list(g.edges(data=True))

[('P auxiliar 1,1', 't', {'cota_min': 0}),
 ('P auxiliar 1,1',
  'P bodega 1,2',
  {'cota_min': 0, 'capacity': 2003, 'weight': 20}),
 ('P auxiliar 2,1', 't', {'cota_min': 0}),
 ('P auxiliar 2,1',
  'P bodega 2,2',
  {'cota_min': 0, 'capacity': 2010, 'weight': 41}),
 ('P auxiliar 3,1', 't', {'cota_min': 0}),
 ('P auxiliar 3,1',
  'P bodega 3,2',
  {'cota_min': 0, 'capacity': 2015, 'weight': 39}),
 ('P auxiliar 4,1', 't', {'cota_min': 0}),
 ('P auxiliar 4,1',
  'P bodega 4,2',
  {'cota_min': 0, 'capacity': 1990, 'weight': 40}),
 ('P auxiliar 5,1', 't', {'cota_min': 0}),
 ('P auxiliar 5,1',
  'P bodega 5,2',
  {'cota_min': 0, 'capacity': 4480, 'weight': 91}),
 ('P auxiliar 6,1', 't', {'cota_min': 0}),
 ('P auxiliar 6,1',
  'P bodega 6,2',
  {'cota_min': 0, 'capacity': 4475, 'weight': 24}),
 ('P auxiliar 7,1', 't', {'cota_min': 0}),
 ('P auxiliar 7,1',
  'P bodega 7,2',
  {'cota_min': 0, 'capacity': 2010, 'weight': 19}),
 ('P auxiliar 1,2', 't', {'cota_min': 0}),
 ('P auxiliar 1,2',
  'P b

In [275]:
result = nx.min_cost_flow(g, demand='demand', capacity='capacity', weight='weight')

NetworkXUnfeasible: no flow satisfies all node demands